# Reuters Graph version

### Sources
- [Philipp tutorial](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html)
- [pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
- [nltk docs](https://www.nltk.org/book/ch02.html)

In [1]:
import torch
import numpy as np

from data_prep.reuters_graph import R8Graph, R52Graph

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

[nltk_data] Downloading package reuters to /home/mat/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [12]:
r8 = R8Graph(device, train_doc=1000)

Prepare Reuters dataset
Compute tf.idf
Compute PMI scores
Generate edges
Generate masks
Generate feature matrix
Features mtx is 0.6030953640000001 GBs in size


In [13]:
print(np.unique(r8.data.y[r8.data.train_mask].cpu(), return_counts=True))
print(np.unique(r8.data.y[r8.data.val_mask].cpu(), return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([268,  41, 451,  15,  31,  43,  30, 121]))
(array([0, 1, 2, 4, 5, 6, 7]), array([30,  4, 49,  5,  3,  3,  6]))


In [14]:
print(sum(r8.data.val_mask * r8.data.train_mask * r8.data.test_mask))
print(sum(r8.data.train_mask))
print(sum(r8.data.val_mask))
print(sum(r8.data.test_mask))

tensor(0, device='cuda:0')
tensor(1000, device='cuda:0')
tensor(100, device='cuda:0')
tensor(100, device='cuda:0')


In [5]:
# GraphConv, GATConv
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(len(r8.iton), 200)
        self.conv2 = GCNConv(200, 8)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

def eval(model, data, mask):
    _, pred = model(data).max(dim=1)
    #print(pred[mask])
    #print(data.y[mask])
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum()
    print('Accuracy: {:.4f}'.format(acc))

In [6]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)#, weight_decay=5e-4)
r8.data.to(device)

Data(edge_attr=[1203414], edge_index=[2, 1203414], test_mask=[12012], train_mask=[12012], val_mask=[12012], x=[12012, 12012], y=[12012])

In [7]:
from tqdm.notebook import tqdm

model.train()
for epoch in tqdm(range(40)):
    optimizer.zero_grad()
    out = model(r8.data)
    # We might want to use the "weight" parameter for the loss with unbalanced dataset
    # since with a low learning rate the model just assigns every doc to class "earn"
    # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    loss = F.cross_entropy(out[r8.data.train_mask], r8.data.y[r8.data.train_mask])
    print('Loss:', loss.item())
    loss.backward()
    optimizer.step()
    print(np.unique(out[r8.data.train_mask].max(dim=1)[1].detach().cpu().numpy(), return_counts=True))
    #print(np.unique(r8.data.y[r8.data.train_mask].detach().cpu().numpy(), return_counts=True))
    eval(model, r8.data, r8.data.val_mask)

Loss: 2.079538345336914
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([ 12,  18,  48,  35,  84, 674,  12, 117]))
Accuracy: 0.4700
Loss: 1.5973663330078125
(array([2]), array([1000]))
Accuracy: 0.4700
Loss: 1.3868449926376343
(array([2]), array([1000]))
Accuracy: 0.4700
Loss: 1.3185598850250244
(array([2]), array([1000]))
Accuracy: 0.7100
Loss: 1.1617223024368286
(array([0, 2]), array([384, 616]))
Accuracy: 0.7500
Loss: 1.0125423669815063
(array([0, 2, 7]), array([466, 533,   1]))
Accuracy: 0.7500
Loss: 0.8748384118080139
(array([0, 2, 7]), array([458, 540,   2]))
Accuracy: 0.7600
Loss: 0.7858579754829407
(array([0, 2, 7]), array([460, 539,   1]))
Accuracy: 0.7700
Loss: 0.7126417756080627
(array([0, 2, 7]), array([482, 515,   3]))
Accuracy: 0.7900
Loss: 0.6526361107826233
(array([0, 2, 7]), array([463, 514,  23]))
Accuracy: 0.8200
Loss: 0.5878263115882874
(array([0, 2, 7]), array([364, 517, 119]))
Accuracy: 0.8000
Loss: 0.5344322323799133
(array([0, 2, 7]), array([288, 518, 194]))
Accuracy: 0

In [8]:
eval(model, r8.data, r8.data.test_mask)

Accuracy: 0.9300
